In [ ]:
import numpy as np
from scipy.optimize import fsolve
import math

In [ ]:
# install
!pip install pycodestyle pycodestyle_magic

# load
%load_ext pycodestyle_magic

In [ ]:
slip = 0.022
mu = lambda cn: 0.75-(0.75*(math.exp(0.3*cn*slip))) - (1.2/cn) + 0.04
#solving(mu, end=100, parts=10000)
#Cn, wheel numeric
print(fsolve(mu,0.7973))

In [6]:
#codes to help develope tractor text parser
from tractorDatabaseParsing import *
import tractorConstructLib as tcl

logging.basicConfig(level=logging.DEBUG,
#filename= 'tractorDataDownloadLog.txt',
format='%(asctime)s - %(levelname)s - %(message)s')

def randomTractorDict():
    with open("PDF URLs.json") as jsonFile:
        urls = jsonFile.readlines()
        randomURL = random.choice(urls).strip()
        print("This is the random url", randomURL)
        #hopefully a tractor url
        tractorDict = onlinePDF2Dict(randomURL)
        return tractorDict

def randomListElement(listy):
    length = len(listy)
    randomInd = random.randint(0, length)
    return listy[randomInd]
def getTitle(strung, subStrung):
    centerInd = strung.index(subStrung)
    frontInd, endInd = [centerInd] * 2
    #find the forward index
    forwardPatt = re.compile(r"\d{2,}\.\d")
    forwardFind = forwardPatt.search(strung, pos=centerInd)
    if forwardFind:
        endInd = forwardFind.start()
    #backwardPatt = re.compile(r"[(]\d+\.?\d*[)]")
    backwardPatt = re.compile(r"PERFORMANCE|\)")
    backwardFind = list(backwardPatt.finditer(strung, endpos=centerInd))
    if backwardFind:
        '''lastMatch = backwardFind[-1]
        print("This is what i found", lastMatch.groups())'''
        poss = [m.end() for m in backwardFind]
        sortedM = [m for _,m in sorted(zip(poss,backwardFind))]
        lastMatch = sortedM[-1]
        frontInd = lastMatch.end()
    print("The indices are", frontInd, centerInd, endInd)
    title = strung[frontInd:endInd]
    return title
def pullOuts(nums, strang=""):
    #gets all brackets in string and calculate if brackets are legit
    #if legit(calculated power == power), store it
    checkLength = 5
    length = len(nums)
    totalCount = 0
    allNums = []
    for i, brac in enumerate(nums[:-checkLength]):
        subNums = nums[i:i+checkLength]
        #power, pull, speed = subNums[:3]
        checkTup = subNums[:checkLength]
        power, pull, speed, fc1, fc2 = checkTup
        calcPower = pull * speed/3.6
        diff = abs(calcPower - power)
        try:
            ratio = diff/min(power, calcPower)
        except:
            continue
        if ratio < (1/100):
            allNums.append(checkTup)
            #separate
            if strang:
                #preNum = allNums[-1][0]
                preNum = allNums[0][-1]
                title = getTitle2(strang, preNum, checkTup[0])
                print("This is title.", str(title))
            totalCount += 1
            #print(f"This is power {power}, pull {pull}, speed {speed}, and ratio {ratio * 100}%.")
            print(f"This is power {power}, pull {pull}, speed {speed}, fc1 {fc1}, fc2 {fc2},  and ratio {ratio * 100}%.")
    print(f"Total count is {totalCount}.")
    return allNums
def getTitle2(strung, preNum, postNum):
    started = False
    holdTitle = []
    pieces = re.split(r"\n+", strung)
    for piece in pieces:
        #if str(preNum) in piece:
        #if str(int(preNum)) in piece:
        preM = re.search(r"\b" + str(preNum) + r"\b", piece)
        if preM:
            print("preM", preM.group())
            started = True
            holdTitle = []
        elif started:
            #if str(postNum) in piece:
            #if str(int(postNum)) in piece:
            postM = re.search(r"\b" + str(postNum) + r"\b", piece)
            if postM:
                print("postM", postM.group())
                break
            else:
                holdTitle.append(piece)
    ansTitle = " ".join(holdTitle)
    return ansTitle
def getTitle3(strung, preNum, postNum):
    started = False
    pieces = re.split(r"\n+", strung)
    st = 0
    nd = len(strung)
    for piece in pieces:
        preM = re.search(r"\b" + str(int(preNum)) + r"\b", piece)
        if preM:
            st = strung.index(pieces)# + preM.end()
            print("preM", preM.group())
            started = True
        elif started:
            postM = re.search(r"\b" + str(int(postNum)) + r"\b", piece)
            if postM:
                nd = strung.index(pieces)# + postM.start()
                print("postM", postM.group())
                break
    ansTitle = strung[st:nd]
    return ansTitle
def getTitle4(streng, listsOfNums, startIndex=0):
    import itertools
    dbPerfsTitles = []
    lastIndices = []
    for j, listOfNums in enumerate(listsOfNums):
        #power, pull, speed
        numTup = listOfNums[:3]
        strTup = [str(num) for num in numTup]
        strMatches = []
        #get all match objects for each number in tup
        for strNum in strTup:
            listy = list(re.finditer(strNum, streng))
            strMatches.append(listy)
        itertools.product(strMatches)
        #default title name is "" if didnt get all matches
        if not all(strMatches):
            dbPerfsTitle.append("")
            continue
        #continue to checking indices
        bestIndices = []
        bestScore = len(streng)
        for strMatchs in strMatches:
            indices = [strMatch.pos for strMatch in strMatchs]
            series = [indic - indices[i+1] for i, indic in enumerate(indices[:-1])]
            score = sum(series)
            if bestIndices:
                if all([num > 0 for num in series]):
                    if score < bestScore:
                        bestScore = score
                        bestIndices = indices
            else:
                bestIndices = indices
        #now, get title
        if j == 0:
            st = 0
        else:
            st = lastIndices[-1]
        nd = bestIndices[0]
        title = streng[st:nd]
        dbPerfsTitles.append(title)
        #save over
        lastIndices = bestIndices
    return dbPerfsTitles

def mane():
    options = 2
    # get a list of tractor from text file and do stuff
    st = time.monotonic()
    strung = randomTractorDict()["text"]
    #bracketNumberPattern = r"\(\d+\.?\d*\)"
    bracketNumberPattern = r"[(]\d+\.?\d*[)]"
    #bracketNumberPattern = r"\((\d+|\d+\.\d+)\)"
    #bracketNumberPattern = r"(\([0-9.]\))"
    #gearPattern = "Gear"
    #gearPattern = r"\d+(st|nd|rd|th)+\([A-D]\d+\)Gear"
    #gearPattern = r"1st|2nd|3rd|\d{1,2}th"
    #gearPattern = r"1st|2nd|3rd|[1-3]?[0-9]th"
    gearPattern = r"\w{5,}"
    #gearPattern = r"\w+"
    drawbarPattern = r"DRAWBAR\s*PERFORMANCE"
    
    #main logic
    if options == 1:
        #write file for one tractor
        oneTractorFileName = "One tractor info.txt"
        #deleteFileIfExist(oneTractorFileName)
        with open(oneTractorFileName, 'w+') as file:
            file.write(strung)
    elif options == 2:
        numz = string2BracketNums(strung)
        titz = getTitle4(strung, numz)
        for tit in titz:
            print(tit)
            print("XXX")
    elif options == 6:
        #dbPerfs = re.split(r"DRAWBAR\s+PERFORMANCE", strung)
        dbPerfs = re.split(r"DRAWBAR", strung)
        allBracketNums = []
        
        for dbPerf in dbPerfs[2:]:
            #dbPerf = dbPerf.lower()
            mostFreqWord = tcl.mostAbundantWord(dbPerf)
            #perfTests = re.split(r"(Gear)|(Range)|(mode)", dbPerf)
            perfTests = re.split(mostFreqWord, dbPerf)
            print("This is the most frequent word in the drawbar performance. ", mostFreqWord)
            print(len(perfTests)-1, ": This is how many Performance tests.")
            
            for i, perfTest in enumerate(perfTests[1:]):
                if perfTest == None:
                    continue
                eachFloat = tcl.extractBracketNumberFromText(perfTest)
                title = getTitle(strung, dbPerf)
                if eachFloat:
                    title = getTitle2(strung, allBracketNums[-1][0], eachFloat[0])
                    print(title)
                    allBracketNums.append(eachFloat)
                if eachFloat:
                    print(eachFloat, "all viable brackets")
                    print("Title XXX", title, "Title end XXX")
        with open("tractorass.txt", "w") as fil:
            fil.writelines(strung)
            json.dump({"Strang":strung}, fil)
            
    #print time
    print("done", time.monotonic() - st)
    

mane()

2020-02-17 22:48:22,352 - DEBUG - Starting new HTTPS connection (1): tractortestlab.unl.edu:443


This is the random url https://tractortestlab.unl.edu/documents/MF5445.pdf


2020-02-17 22:48:22,876 - DEBUG - https://tractortestlab.unl.edu:443 "GET /documents/MF5445.pdf HTTP/1.1" 200 100977


This is power 45.1, pull 16.4, speed 9.89, fc1 0.353, fc2 2.41,  and ratio 0.10111223458039215%.
This is power 34.9, pull 12.3, speed 10.2, fc1 0.41, fc2 2.07,  and ratio 0.14347202295553593%.
This is power 23.5, pull 8.2, speed 10.33, fc1 0.53, fc2 1.6,  and ratio 0.12529550827421349%.
This is power 35.2, pull 12.4, speed 10.24, fc1 0.343, fc2 2.48,  and ratio 0.20202020202019322%.
This is power 23.9, pull 8.3, speed 10.4, fc1 0.43, fc2 1.98,  and ratio 0.32543003254300956%.
This is power 38.4, pull 38.6, speed 3.58, fc1 0.404, fc2 2.1,  and ratio 0.0376298955046529%.
This is power 45.2, pull 35.1, speed 4.64, fc1 0.351, fc2 2.42,  and ratio 0.08849557522123705%.
This is power 46.7, pull 35.8, speed 4.69, fc1 0.326, fc2 2.61,  and ratio 0.12983764338722284%.
This is power 48.0, pull 28.0, speed 6.18, fc1 0.317, fc2 2.69,  and ratio 0.138888888888881%.
This is power 48.2, pull 26.4, speed 6.58, fc1 0.313, fc2 2.71,  and ratio 0.11065006915628103%.
This is power 49.0, pull 21.4, speed 8

In [ ]:
#obtain metadata off of online pdf
import PyPDF2

url = r'https://tractortestlab.unl.edu/John%20Deere%206170R.pdf'
#url = r'https://tractortestlab.unl.edu/documents/AGO%209755%20Cummins.pdf'
url = r'https://tractortestlab.unl.edu/documents/70-79.pdf'
url = r'https://tractortestlab.unl.edu/documents/TYM_T603.pdf'
pdfName = f"{randomString()}.pdf"
onlinePDF2File(url, pdfName)

with open(pdfName, 'rb') as pdfFile:
    reader = PyPDF2.PdfFileReader(pdfFile)
    metadata = reader.getDocumentInfo()
    print(metadata)
    
os.remove(pdfName)

In [ ]:
#convert pdf url to image
from tractorDatabaseParsing import *
import pdf2image, cv2
from PIL import Image

url = r'https://tractortestlab.unl.edu/John%20Deere%206170R.pdf'
randomStrung = randomString()
pdfName = f"{randomStrung}.pdf"
onlinePDF2File(url, pdfName)

#with open(pdfName, 'rb') as pdfFile:
time.sleep(3)
pdf2image.convert_from_path(pdfName)
os.remove(pdfName)

In [ ]:
#obtain metadata off of local pdf
import PyPDF2
pdfName = "John Deere 6170R.pdf"
pdfName = "Books/Coroutines.pdf"

with open(pdfName, 'rb') as pdfFile:
    reader = PyPDF2.PdfFileReader(pdfFile)
    metadata = reader.getDocumentInfo()
    print(metadata)
    

In [ ]:
#tika parser
from tika import parser

pdfName = "Books/Coroutines.pdf"
parsedPDF = parser.from_file(pdfName)
metadata = parsedPDF.metadata


In [ ]:
#complete processes to obtain tractor dictionaries from the internet
from tractorDatabaseParsing import *

st = time.monotonic()
urls = motherPageSubURLs()
print('obtained urls')
pdfURLs = url_pdf_filter(urls)
print('filtered to pdf urls')
tractorDicts = tractors_pdf_dict(pdfURLs)
print('Amount of tractors', len(tractorDicts))
print('obtained dictionaries of pdfs')
et = time.monotonic()
print('Duration time, ', et-st)

In [ ]:
#get all pdf urls into a file
from tractorDatabaseParsing import *

st = time.monotonic()
urls = motherPageSubURLs()
print('obtained urls')
pdfURLs = url_pdf_filter(urls)
print('filtered to pdf urls')
with open("PDF URLs.json", mode="w+") as jsonFile:
    for url in pdfURLs:
        jsonFile.write(url+"\n")

In [ ]:
#print out text/dictionary of tractor url
from tractorDatabaseParsing import *

url = r"https://tractortestlab.unl.edu/Case%20IH%20Puma%20240%20FT4.pdf"
tractorDict = onlinePDF2Dict(url)
print(tractorDict["text"])

In [ ]:
import sys

print(len(pdfURLs))
print(len(tractorDicts))
finalDicts = [x for x in tractorDicts if bool(x)]
print(len(finalDicts))
print(sys.getsizeof(finalDicts[5]))
print(finalDicts[866])

In [ ]:
jsonSubUrlList2Text()